In [4]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#to keep things in order, and to avoid to copy and paste everytime our functions if we want to use them in more than one folder,
#we can temporarily use this library. 
import sys

#in this way Python will search the implementations also in the path '../HelperFunctions'
sys.path.insert(0, '../HelperFunctions')
sys.path.insert(0, '../pre-processing/Clean_Data/')
sys.path.insert(0, '../Logit')

In [6]:
from proj1_helpers import *
from common_functions import *
from counters import *
from remove import *
from replace import *
from regressors import batch_iter
from functions_logistic import *

In [7]:
yb, input_data, ids = load_csv_data("../data/train.csv", sub_sample=False)

In [8]:
#this will surely be deleted, in this way we are sure that original_data is the original version of the data and we don't have
#to load them again
from copy import deepcopy
originalData = deepcopy(input_data)
originalY = deepcopy(yb)
print(yb)


[ 1. -1. -1. ...  1. -1. -1.]


# Functions from lab 5

In [9]:
#Creation of tx

input_data = deepcopy(originalData)
print(input_data.shape)

#Clean the dataset
numInvalidValues = countInvalid(input_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
input_data = replaceWithZero(input_data,-999,idxCols)

# Stocking the indexes of columns to remove
#idxCols = np.where(countInvalid(input_data,-999)>0)[0]
#input_data=removeColumns(input_data,0)
#print(input_data.shape)

#standardize
tx,_,_ = standardize(input_data)

#add ones
#tx = np.c_[np.ones((yb.shape[0], 1)), tx]
y = yb
#print(tx.shape)

(250000, 30)


In [10]:
#To understand better standardization
A=[[1.3,4.7,8],[0.4,5,6],[2,5,1.2]]
print(A)
print(np.mean(A,axis=0))
print(A-np.mean(A,axis=0))
a=A-np.mean(A,axis=0)
b=np.std(A,axis=0)
print('std =',b)
print(a/np.std(A,axis=0))

[[1.3, 4.7, 8], [0.4, 5, 6], [2, 5, 1.2]]
[1.23333333 4.9        5.06666667]
[[ 0.06666667 -0.2         2.93333333]
 [-0.83333333  0.1         0.93333333]
 [ 0.76666667  0.1        -3.86666667]]
std = [0.65489609 0.14142136 2.85345794]
[[ 0.10179732 -1.41421356  1.02799249]
 [-1.2724665   0.70710678  0.32708852]
 [ 1.17066918  0.70710678 -1.35508101]]


In [ ]:
#first tries with gradient descent

#Y, TX = sample_data(y, tx, 10, 150000)
max_iter = 100000
w = np.ones(tx.shape[1])
gamma = 10000
loss0 = 0

for iter in range(max_iter):
    w_old = w
    loss, w = learning_by_gradient_descent(y, tx, w, gamma)
    print('w =',w)
    
    if iter%100==0:
        #gamma=0.55*gamma
        w_new = w
        print('iter = ', iter)
        #print(f'Iter = {iter}, Loss = {loss}, |w_old - w_new| = {np.linalg.norm(w_old-w_new)}')




[9.99964004e-01 7.17746986e-01 9.97774477e-01 ... 9.51513627e-03
 1.21004458e-04 9.62364309e-04]
logistic loss:  1517221.8143192923
logistic loss corection only:  8059047.825479169
w = [ 1.19646931e+07 -7.63094089e+08 -2.74885465e+08 -2.48389333e+08
 -1.09840309e+08 -1.27675713e+07 -2.51572509e+08  2.74338611e+08
 -3.85924426e+08 -4.47691227e+08 -6.35997594e+08  1.23449145e+08
 -7.74947729e+07  2.31451424e+08 -1.55818545e+08 -6.51484190e+07
 -5.06381694e+08 -1.58066303e+08 -2.67771458e+07 -3.87551018e+08
 -5.27945146e+07 -4.41630101e+08 -5.40739418e+08 -3.50639025e+08
 -1.46500871e+08 -4.70662105e+07 -4.57160688e+08 -4.72738516e+07
 -4.97217240e+07 -4.67421683e+08]
iter =  0
[0. 1. 0. ... 1. 1. 1.]
logistic loss:  4445886.537178324
logistic loss corection only:  8059047.825479169
w = [ 5.52979284e+08 -1.45796478e+09 -1.99836927e+08  9.10592238e+08
  9.68634097e+08  9.96721477e+08 -7.67576414e+08 -1.00855026e+08
 -4.03639746e+07  7.14956951e+08 -7.49537444e+08  1.12702566e+09
  9.645304

w = [ 1.06178051e+09 -2.71157280e+09 -1.18160798e+09  1.16260210e+09
  1.38916866e+09  2.54657630e+09 -1.82818565e+09  1.41361309e+09
 -8.99338084e+08  4.90418946e+07 -1.22880750e+09  1.11330418e+09
  1.40759629e+09  2.18857254e+09 -4.11969034e+07 -5.18072029e+07
  5.62345141e+08 -3.13322196e+06  1.76298917e+07  5.80654256e+07
  1.80608678e+07 -2.82690673e+08 -9.24632537e+08  1.66388360e+08
  3.19072808e+06 -1.17280459e+07 -5.91112969e+08 -2.01999340e+06
 -3.40368647e+07 -5.69134418e+08]
[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2232195.0305502163
logistic loss corection only:  8059047.825479169
w = [ 1.09973066e+09 -2.79287526e+09 -1.25587313e+09  1.21438001e+09
  1.43097027e+09  2.66756607e+09 -1.87799190e+09  1.48608433e+09
 -9.22386948e+08  4.80338504e+07 -1.26032345e+09  1.14322927e+09
  1.44666492e+09  2.26903752e+09 -4.30305356e+07 -5.20740227e+07
  6.03775347e+08 -3.11438762e+06  1.91869155e+07  7.04779177e+07
  1.91781285e+07 -2.97146006e+08 -9.61716973e+08  1.68924016e+08
  4.0

w = [ 1.62403489e+09 -3.87729912e+09 -2.18031182e+09  1.87503094e+09
  1.96512746e+09  4.22874209e+09 -2.51315606e+09  2.40470599e+09
 -1.24172949e+09  3.87390707e+07 -1.73833314e+09  1.56586921e+09
  1.95489881e+09  3.33053670e+09 -6.47534660e+07 -5.74333111e+07
  1.07866407e+09 -6.98971040e+05  3.39807493e+07  1.83612478e+08
  3.21708416e+07 -4.75104304e+08 -1.39158817e+09  2.15832372e+08
  6.41064095e+06 -4.22069083e+06 -9.29968435e+08  9.89249600e+06
 -4.19777478e+07 -9.58647826e+08]
[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2215271.0300117107
logistic loss corection only:  8059047.825479169
w = [ 1.66564583e+09 -3.96192126e+09 -2.24902335e+09  1.92521060e+09
  2.00603160e+09  4.34849868e+09 -2.56229456e+09  2.47421419e+09
 -1.26737420e+09  3.82085219e+07 -1.77760852e+09  1.59922900e+09
  1.99445055e+09  3.41266560e+09 -6.60157026e+07 -5.80376128e+07
  1.11271678e+09 -2.66337792e+05  3.49893124e+07  1.90211165e+08
  3.30250786e+07 -4.88353099e+08 -1.42262850e+09  2.20131899e+08
  6.7

[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2209855.3498393884
logistic loss corection only:  8059047.825479169
w = [ 2.24625206e+09 -5.15834582e+09 -3.20435991e+09  2.62473026e+09
  2.57836372e+09  6.02186954e+09 -3.25655891e+09  3.42979569e+09
 -1.63230986e+09  3.01339579e+07 -2.33920081e+09  2.07313618e+09
  2.55597057e+09  4.56289716e+09 -8.75650280e+07 -7.03642455e+07
  1.57508502e+09  4.72535722e+06  4.83524851e+07  2.67242780e+08
  4.35194049e+07 -6.71663751e+08 -1.84605529e+09  2.83932346e+08
  8.77457140e+06 -1.01400055e+06 -1.29009558e+09  1.57891369e+07
 -5.64905433e+07 -1.36234760e+09]
[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2209694.1688818787
logistic loss corection only:  8059047.825479169
w = [ 2.28800185e+09 -5.24424406e+09 -3.27164113e+09  2.67453281e+09
  2.61919089e+09  6.14122852e+09 -3.30668559e+09  3.49738089e+09
 -1.65864580e+09  2.96013187e+07 -2.37984285e+09  2.10710927e+09
  2.59645893e+09  4.64524072e+09 -8.91333772e+07 -7.13472092e+07
  1.60768804e+09  4.9006042

[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2207405.399285243
logistic loss corection only:  8059047.825479169
w = [ 2.83469197e+09 -6.36423717e+09 -4.13694406e+09  3.32190647e+09
  3.14907369e+09  7.69189228e+09 -3.96279890e+09  4.37049911e+09
 -2.00619302e+09  2.30995426e+07 -2.90964955e+09  2.55301827e+09
  3.12439846e+09  5.71589685e+09 -1.08869208e+08 -8.62640883e+07
  2.02804602e+09  5.25256324e+06  6.27653669e+07  3.31232964e+08
  5.50795997e+07 -8.52191909e+08 -2.26257175e+09  3.53389197e+08
  1.12774084e+07 -2.82399430e+06 -1.62967374e+09  1.91462369e+07
 -6.86429397e+07 -1.73626314e+09]
[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2207373.163093741
logistic loss corection only:  8059047.825479169
w = [ 2.87687008e+09 -6.45058395e+09 -4.20300509e+09  3.37165775e+09
  3.18990420e+09  7.81116206e+09 -4.01352252e+09  4.43732656e+09
 -2.03303551e+09  2.26314036e+07 -2.95050816e+09  2.58740403e+09
  3.16531606e+09  5.79828044e+09 -1.10398952e+08 -8.73982332e+07
  2.06019471e+09  5.20788575e

...

w = [ 1.94957387e+10 -4.12847459e+10 -2.97505488e+10  2.30882339e+10
  1.93580958e+10  5.50244638e+10 -2.46848461e+10  3.04665880e+10
 -1.29234538e+10 -2.06452867e+08 -1.93149427e+10  1.64299869e+10
  1.98552006e+10  3.84126794e+10 -6.96979035e+08 -5.58004286e+08
  1.44510774e+10 -6.18799991e+06  4.45568011e+08  1.87196392e+09
  3.48043446e+08 -5.88579677e+09 -1.40393517e+10  2.42155514e+09
  6.50250269e+07 -3.75164732e+07 -1.13483302e+10  1.48784931e+08
 -4.42117856e+08 -1.22802538e+10]
[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2201248.2867083773
logistic loss corection only:  8059047.825479169
w = [ 1.95375834e+10 -4.13727133e+10 -2.98145163e+10  2.31378874e+10
  1.93988136e+10  5.51433171e+10 -2.47371548e+10  3.05319241e+10
 -1.29510128e+10 -2.07077500e+08 -1.93562457e+10  1.64649328e+10
  1.98974120e+10  3.84948244e+10 -6.98475972e+08 -5.59195038e+08
  1.44821370e+10 -6.18477739e+06  4.46546585e+08  1.87576529e+09
  3.48788004e+08 -5.89841433e+09 -1.40690051e+10  2.42675405e+09
  6.5

w = [ 2.00810739e+10 -4.25162888e+10 -3.06466283e+10  2.37834692e+10
  1.99282246e+10  5.66884582e+10 -2.54173600e+10  3.13810524e+10
 -1.33091232e+10 -2.15207847e+08 -1.98930960e+10  1.69193210e+10
  2.04462637e+10  3.95625566e+10 -7.17757048e+08 -5.74822877e+08
  1.48857895e+10 -6.02055556e+06  4.59416918e+08  1.92527544e+09
  3.58482987e+08 -6.06237881e+09 -1.44544314e+10  2.49438605e+09
  6.72568620e+07 -3.85287752e+07 -1.16913233e+10  1.54409731e+08
 -4.55784126e+08 -1.26513784e+10]
[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2201312.759091381
logistic loss corection only:  8059047.825479169
w = [ 2.01228527e+10 -4.26042367e+10 -3.07106667e+10  2.38331197e+10
  1.99689512e+10  5.68073104e+10 -2.54697060e+10  3.14463633e+10
 -1.33366691e+10 -2.15840542e+08 -1.99343852e+10  1.69542662e+10
  2.04885030e+10  3.96446905e+10 -7.19247707e+08 -5.76029432e+08
  1.49168489e+10 -6.02903283e+06  4.60413246e+08  1.92907528e+09
  3.59235684e+08 -6.07500357e+09 -1.44840848e+10  2.49957479e+09
  6.74

[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2201248.2867083773
logistic loss corection only:  8059047.825479169
w = [ 2.07083042e+10 -4.38356159e+10 -3.16066984e+10  2.45283818e+10
  2.05391975e+10  5.84712980e+10 -2.62022940e+10  3.23607432e+10
 -1.37221843e+10 -2.24603283e+08 -2.05123865e+10  1.74440235e+10
  2.10799276e+10  4.07944516e+10 -7.40059568e+08 -5.92440521e+08
  1.53516498e+10 -6.14450130e+06  4.74216499e+08  1.98240031e+09
  3.69594946e+08 -6.25163189e+09 -1.48991103e+10  2.57236966e+09
  6.96183392e+07 -3.94646820e+07 -1.20587316e+10  1.60299866e+08
 -4.70185554e+08 -1.30490249e+10]
[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2201312.759091381
logistic loss corection only:  8059047.825479169
w = [ 2.07500874e+10 -4.39235534e+10 -3.16707274e+10  2.45780327e+10
  2.05799422e+10  5.85901551e+10 -2.62546453e+10  3.24260482e+10
 -1.37497226e+10 -2.25233318e+08 -2.05536707e+10  1.74789888e+10
  2.11221976e+10  4.08765764e+10 -7.41528005e+08 -5.93616836e+08
  1.53827096e+10 -6.17585640

[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2201216.0505168755
logistic loss corection only:  8059047.825479169
w = [ 2.12935872e+10 -4.50669254e+10 -3.25026834e+10  2.52235214e+10
  2.11095766e+10  6.01352728e+10 -2.69349592e+10  3.32752186e+10
 -1.41076753e+10 -2.33453652e+08 -2.10904635e+10  1.79337032e+10
  2.16716458e+10  4.19442073e+10 -7.60656750e+08 -6.08971137e+08
  1.57863203e+10 -6.43553285e+06  4.87856205e+08  2.03576755e+09
  3.80049892e+08 -6.42826544e+09 -1.53141118e+10  2.64505926e+09
  7.16296996e+07 -4.05390765e+07 -1.24016243e+10  1.65840803e+08
 -4.83478628e+08 -1.34202249e+10]
[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2201248.2867083773
logistic loss corection only:  8059047.825479169
w = [ 2.13354294e+10 -4.51548846e+10 -3.25666354e+10  2.52731758e+10
  2.11503146e+10  6.02541264e+10 -2.69872744e+10  3.33405531e+10
 -1.41352083e+10 -2.34082967e+08 -2.11317650e+10  1.79686729e+10
  2.17139070e+10  4.20263459e+10 -7.62140977e+08 -6.10158304e+08
  1.58173696e+10 -6.4491010

[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2201216.0505168755
logistic loss corection only:  8059047.825479169
w = [ 2.19207476e+10 -4.63862678e+10 -3.34624809e+10  2.59683307e+10
  2.17206754e+10  6.19180902e+10 -2.77198662e+10  3.42550780e+10
 -1.45206473e+10 -2.42946735e+08 -2.17099159e+10  1.84583665e+10
  2.23056030e+10  4.31761491e+10 -7.82766058e+08 -6.26801675e+08
  1.62519544e+10 -6.72370907e+06  5.02439211e+08  2.09298839e+09
  3.91247231e+08 -6.61748047e+09 -1.57587652e+10  2.72293383e+09
  7.38276647e+07 -4.16860881e+07 -1.27690077e+10  1.71841600e+08
 -4.97769200e+08 -1.38179465e+10]
[0. 1. 0. ... 0. 1. 0.]
logistic loss:  2201216.0505168755
logistic loss corection only:  8059047.825479169
w = [ 2.19625041e+10 -4.64742181e+10 -3.35265238e+10  2.60179798e+10
  2.17614153e+10  6.20369434e+10 -2.77722162e+10  3.43203854e+10
 -1.45481733e+10 -2.43588742e+08 -2.17512077e+10  1.84933482e+10
  2.23478686e+10  4.32582688e+10 -7.84220180e+08 -6.27991084e+08
  1.62829897e+10 -6.7470284

Let's try to generate a submission

In [ ]:
WWW = w 

In [ ]:
xxx, test_data, ids = load_csv_data("../data/test.csv", sub_sample=False)

In [ ]:
#we need to preprocess the test_data as we preprocessed the train data

numInvalidValues = countInvalid(test_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
test_data = replaceWithZero(test_data,-999,idxCols)
txTest,_,_ = standardize(test_data)
txTest = np.c_[np.ones((xxx.shape[0], 1)), txTest]

In [ ]:
y_pred = predict_labels(WWW, txTest)

In [ ]:
create_csv_submission(ids, y_pred, 'submission.csv')

# Same thing with PCA

In [ ]:
sys.path.insert(0, '../pre-processing/PCA/')

In [ ]:
from pca_functions import PCAWithCovariance

input_data = deepcopy(originalData)
y = deepcopy(originalY)
print(input_data.shape)
print(y.shape)
numInvalidValues = countInvalid(input_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
input_data = replaceWithZero(input_data,-999,idxCols)
input_data,_,_ = standardize(input_data)

In [ ]:
from pca_functions import PCAWithCovariance

input_data = deepcopy(originalData)
y = deepcopy(originalY)
print(input_data.shape)
print(y.shape)
input_data=removeColumns(input_data,0)
print(input_data.shape)
input_data,_,_ = standardize(input_data)

In [ ]:
#Generating the principal components

_,eV = PCAWithCovariance(input_data)

N = 7 #num p. components
components = np.empty(input_data.shape[0])
for i in range(N):
    components = np.c_[components, input_data.dot(eV[:,i])]
    
print(components.shape)
print(components[:,0])
tx = np.c_[np.ones(input_data.shape[0]), components]
print(tx.shape)
print(tx[:,0])
print(tx[:,1])

In [ ]:
Y, TX = sample_data(y, tx, 1, 70000)
TX.shape

In [ ]:
max_iter = 1000
w = np.ones(tx.shape[1])
gamma = 10000
loss0 = 0
for iter in range(max_iter):
    wold = w
    loss, w = learning_by_gradient_descent(yb, tx, w, gamma)
    #gamma=0.55*gamma
    if iter%10==0:
        wnew = w
        print(f'Iter = {iter}, Loss = {loss}, |wold - wnew| = {np.linalg.norm(wold-wnew)}')
print('final loss =',loss)

In [ ]:
# generate submission with PCA
xxx, test_data, ids = load_csv_data("../data/test.csv", sub_sample=False)



In [ ]:
numInvalidValues = countInvalid(test_data,-999)
idxCols = np.where(numInvalidValues>0)[0]
test_data = replaceWithZero(test_data,-999,idxCols)
test_data,_,_ = standardize(test_data)

components = np.empty(test_data.shape[0])
for i in range(N):
    components = np.c_[components, test_data.dot(eV[:,i])]
    
#tx_test = np.c_[np.ones(test_data.shape[0]), components]
tx_test = components

y_pred = predict_labels(w, tx_test)
create_csv_submission(ids, y_pred, 'submission.csv')